In [ ]:
import laspy
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, box
from pathlib import Path
import contextily as cx          # pip install contextily
from pyproj import CRS
import pprint

def plot_las_file(file_path, every_nth=1000):

    las  = laspy.read(file_path)
    xyz  = las.xyz[::every_nth]          
    x, y, z = xyz.T                      


    crs_las = las.header.parse_crs(prefer_wkt=True)  # prefer WKT VLR (LAS 1.4) over GeoTIFF keys (LAS 1.0-1.3)
    print(crs_las)


    gdf_pts  = gpd.GeoDataFrame({"elev": z}, geometry=gpd.points_from_xy(x, y), crs=crs_las)

    xmin, ymin, _ = las.header.mins
    xmax, ymax, _ = las.header.maxs

    gdf_pts_web  = gdf_pts.to_crs(epsg=3857)
    #gdf_bbox_web = gdf_bbox.to_crs(epsg=3857)

    fig, ax = plt.subplots()


    gdf_pts_web.plot(ax=ax, column="elev", markersize=2, cmap="viridis", alpha=0.6, legend=True, legend_kwds={"label": "Elevation (m)"})

    #gdf_bbox_web.boundary.plot(ax=ax, edgecolor="red", linewidth=2, label="Bounding box")

    cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)

    ax.set_axis_off()
    ax.set_title(Path(file_path).name, fontsize=12)
    plt.tight_layout()
    plt.show()


def print_header(path):
    with laspy.open(path) as f:
        hdr = f.header
        print("=== CORE HEADER ===")
        print(hdr)                       # as above

        print("\n=== VLRs ===")
        for i, vlr in enumerate(hdr.vlrs, 1):
            print(f"\n[VLR {i}]  user_id={vlr.user_id}  record_id={vlr.record_id}")
            pprint.pprint(vlr.__dict__, width=120)

        if hdr.evlrs:
            print("\n=== EVLRs ===")
            for i, evlr in enumerate(hdr.evlrs, 1):
                print(f"\n[EVLR {i}] user_id={evlr.user_id} record_id={evlr.record_id}")
                pprint.pprint(evlr.__dict__, width=120)

#plot_las_file(path, every_nth=2_000)   # show 1 point out of 2 000


In [ ]:
import os

def check_if_multigeometry(path):
    
    gdf = gpd.read_file(path)

    base_name = os.path.basename(path)

    if len(gdf) > 1:
        print(f"The file {base_name} contains multiple geometry types: {gdf.geom_type.unique()}")
    

folder = r''

file_paths = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.gpkg')]

for f in file_paths:
    check_if_multigeometry(f)
    

In [ ]:
#
path = r""
preprocessed_path = r''

print_header(path)  # print header info
plot_las_file(path, every_nth=2_000) 